# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 10 2022 12:26PM,243885,12,6826069,Doctors Scientific Organica,Released
1,Aug 10 2022 12:21PM,243883,10,0085932331,ISDIN Corporation,Released
2,Aug 10 2022 12:21PM,243883,10,0085932785,ISDIN Corporation,Released
3,Aug 10 2022 12:21PM,243883,10,0085932905,ISDIN Corporation,Released
4,Aug 10 2022 12:21PM,243883,10,0085932908,ISDIN Corporation,Released
5,Aug 10 2022 12:21PM,243883,10,0085932906,ISDIN Corporation,Released
6,Aug 10 2022 12:20PM,243881,19,ACG-2102490904,ACG North America LLC,Released
7,Aug 10 2022 12:20PM,243882,10,0085932904,ISDIN Corporation,Released
8,Aug 10 2022 12:20PM,243880,19,60016163.1,"GUSA Granules USA, Inc.",Released
9,Aug 10 2022 12:20PM,243880,19,60010881,"GUSA Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,243880,Released,2
40,243881,Released,1
41,243882,Released,1
42,243883,Released,5
43,243885,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243880,NaN,NaN,2.0
243881,NaN,NaN,1.0
243882,NaN,NaN,1.0
243883,NaN,NaN,5.0
243885,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243769,0.0,0.0,9.0
243778,27.0,1.0,1.0
243795,15.0,5.0,0.0
243805,0.0,0.0,15.0
243808,32.0,8.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243769,0,0,9
243778,27,1,1
243795,15,5,0
243805,0,0,15
243808,32,8,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243769,0,0,9
1,243778,27,1,1
2,243795,15,5,0
3,243805,0,0,15
4,243808,32,8,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243769,,,9
1,243778,27,1,1
2,243795,15,5,
3,243805,,,15
4,243808,32,8,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 10 2022 12:26PM,243885,12,Doctors Scientific Organica
1,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation
6,Aug 10 2022 12:20PM,243881,19,ACG North America LLC
7,Aug 10 2022 12:20PM,243882,10,ISDIN Corporation
8,Aug 10 2022 12:20PM,243880,19,"GUSA Granules USA, Inc."
10,Aug 10 2022 12:19PM,243879,19,ACG North America LLC
11,Aug 10 2022 12:18PM,243878,10,Bio-PRF
14,Aug 10 2022 12:12PM,243871,20,"ACI Healthcare USA, Inc."
28,Aug 10 2022 12:08PM,243877,19,"GUSA Granules USA, Inc."
29,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 10 2022 12:26PM,243885,12,Doctors Scientific Organica,,,1
1,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,,,5
2,Aug 10 2022 12:20PM,243881,19,ACG North America LLC,,,1
3,Aug 10 2022 12:20PM,243882,10,ISDIN Corporation,,,1
4,Aug 10 2022 12:20PM,243880,19,"GUSA Granules USA, Inc.",,,2
5,Aug 10 2022 12:19PM,243879,19,ACG North America LLC,,,1
6,Aug 10 2022 12:18PM,243878,10,Bio-PRF,,,3
7,Aug 10 2022 12:12PM,243871,20,"ACI Healthcare USA, Inc.",,,14
8,Aug 10 2022 12:08PM,243877,19,"GUSA Granules USA, Inc.",,,1
9,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:26PM,243885,12,Doctors Scientific Organica,1,,
1,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,5,,
2,Aug 10 2022 12:20PM,243881,19,ACG North America LLC,1,,
3,Aug 10 2022 12:20PM,243882,10,ISDIN Corporation,1,,
4,Aug 10 2022 12:20PM,243880,19,"GUSA Granules USA, Inc.",2,,
5,Aug 10 2022 12:19PM,243879,19,ACG North America LLC,1,,
6,Aug 10 2022 12:18PM,243878,10,Bio-PRF,3,,
7,Aug 10 2022 12:12PM,243871,20,"ACI Healthcare USA, Inc.",14,,
8,Aug 10 2022 12:08PM,243877,19,"GUSA Granules USA, Inc.",1,,
9,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:26PM,243885,12,Doctors Scientific Organica,1,,
1,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,5,,
2,Aug 10 2022 12:20PM,243881,19,ACG North America LLC,1,,
3,Aug 10 2022 12:20PM,243882,10,ISDIN Corporation,1,,
4,Aug 10 2022 12:20PM,243880,19,"GUSA Granules USA, Inc.",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:26PM,243885,12,Doctors Scientific Organica,1.0,NaN,NaN
1,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,5.0,NaN,NaN
2,Aug 10 2022 12:20PM,243881,19,ACG North America LLC,1.0,NaN,NaN
3,Aug 10 2022 12:20PM,243882,10,ISDIN Corporation,1.0,NaN,NaN
4,Aug 10 2022 12:20PM,243880,19,"GUSA Granules USA, Inc.",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:26PM,243885,12,Doctors Scientific Organica,1.0,0.0,0.0
1,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,5.0,0.0,0.0
2,Aug 10 2022 12:20PM,243881,19,ACG North America LLC,1.0,0.0,0.0
3,Aug 10 2022 12:20PM,243882,10,ISDIN Corporation,1.0,0.0,0.0
4,Aug 10 2022 12:20PM,243880,19,"GUSA Granules USA, Inc.",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1950875,46.0,22.0,3.0
12,243885,1.0,0.0,0.0
15,975449,45.0,20.0,0.0
16,243865,1.0,0.0,0.0
19,2194768,9.0,17.0,19.0
20,1950567,78.0,18.0,59.0
21,243828,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1950875,46.0,22.0,3.0
1,12,243885,1.0,0.0,0.0
2,15,975449,45.0,20.0,0.0
3,16,243865,1.0,0.0,0.0
4,19,2194768,9.0,17.0,19.0
5,20,1950567,78.0,18.0,59.0
6,21,243828,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,46.0,22.0,3.0
1,12,1.0,0.0,0.0
2,15,45.0,20.0,0.0
3,16,1.0,0.0,0.0
4,19,9.0,17.0,19.0
5,20,78.0,18.0,59.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,46.0
1,12,Released,1.0
2,15,Released,45.0
3,16,Released,1.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,3.0,0.0,0.0,0.0,19.0,59.0,0.0
Executing,22.0,0.0,20.0,0.0,17.0,18.0,1.0
Released,46.0,1.0,45.0,1.0,9.0,78.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,3.0,0.0,0.0,0.0,19.0,59.0,0.0
1,Executing,22.0,0.0,20.0,0.0,17.0,18.0,1.0
2,Released,46.0,1.0,45.0,1.0,9.0,78.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,3.0,0.0,0.0,0.0,19.0,59.0,0.0
1,Executing,22.0,0.0,20.0,0.0,17.0,18.0,1.0
2,Released,46.0,1.0,45.0,1.0,9.0,78.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()